In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab-ml").getOrCreate()


In [2]:
spark

In [3]:
#df = spark.read.parquet("s3://aws-logs-780494741284-us-east-1/dataset/project_graph_data")

In [4]:
#df.take(10)

In [5]:
#df.count()

In [6]:
#df.distinct().count()

In [7]:
df_node = spark.read.parquet("s3://aws-logs-780494741284-us-east-1/dataset/graph_node_domain")

In [8]:
df_node.take(10)

[Row(urls='5.ssbet180.cn', label=1169),
 Row(urls='WWW.000747.COM', label=1152),
 Row(urls='WWW.000747.COM', label=1152),
 Row(urls='WWW.013821.COM', label=1125),
 Row(urls='WWW.013821.COM', label=1125),
 Row(urls='WWW.013821.COM', label=1125),
 Row(urls='WWW.126357.COM', label=1160),
 Row(urls='WWW.126357.COM', label=1160),
 Row(urls='WWW.126357.COM', label=1160),
 Row(urls='WWW.1418000.COM', label=1150)]

In [9]:
df_node.count()

54947

In [10]:
df_node.distinct().count()

8049

In [11]:
df_edge = spark.read.parquet("s3://aws-logs-780494741284-us-east-1/dataset/graph_edges_domain")

In [12]:
df_edge.take(10)

[Row(target='7249', source='2213'),
 Row(target='7295', source='2213'),
 Row(target='3675', source='2213'),
 Row(target='7614', source='2213'),
 Row(target='3831', source='2213'),
 Row(target='5183', source='2213'),
 Row(target='5151', source='2213'),
 Row(target='1002', source='2213'),
 Row(target='2810', source='2213'),
 Row(target='6360', source='2213')]

In [13]:
df_edge.count()

23381

In [14]:
df_edge.distinct().count()

19271

In [15]:
#keep dinstict edges
df_edge=df_edge.distinct()

In [16]:
df_edge.count()

19271

In [17]:
#edge_source_dinst=df_edge.select("source").distinct().show()

In [18]:
#number of distinct source node in dataset
edge_source_num=df_edge.select("source").distinct().count()

In [19]:
edge_source_num

17

In [20]:
#source_dinst=df.select("source").distinct().show()

In [21]:
#number of dinstinct urls in dataset
nodes_num=df_node.select("urls").distinct().count()

In [22]:
nodes_num

8049

In [23]:
#inital value for each distinct url
initial_value=1/nodes_num

In [24]:
initial_value

0.00012423903590508137

In [25]:
from pyspark.sql.functions import *

In [26]:
#add initial value column
df_edge = df_edge.withColumn("initialValue", lit(initial_value))

In [27]:
df_edge.take(10)

[Row(target='6984', source='2213', initialValue=0.00012423903590508137),
 Row(target='6210', source='6537', initialValue=0.00012423903590508137),
 Row(target='6506', source='401', initialValue=0.00012423903590508137),
 Row(target='2116', source='1514', initialValue=0.00012423903590508137),
 Row(target='42', source='401', initialValue=0.00012423903590508137),
 Row(target='4487', source='7565', initialValue=0.00012423903590508137),
 Row(target='7020', source='7565', initialValue=0.00012423903590508137),
 Row(target='4569', source='7565', initialValue=0.00012423903590508137),
 Row(target='5626', source='2213', initialValue=0.00012423903590508137),
 Row(target='5103', source='7565', initialValue=0.00012423903590508137)]

In [28]:
#sort dataset by source id
df_edge=df_edge.sort(col("source"))

In [29]:
df_edge.take(10)

[Row(target='6780', source='1241', initialValue=0.00012423903590508137),
 Row(target='4595', source='1241', initialValue=0.00012423903590508137),
 Row(target='7512', source='1241', initialValue=0.00012423903590508137),
 Row(target='6873', source='1514', initialValue=0.00012423903590508137),
 Row(target='5471', source='1514', initialValue=0.00012423903590508137),
 Row(target='195', source='1514', initialValue=0.00012423903590508137),
 Row(target='5772', source='1514', initialValue=0.00012423903590508137),
 Row(target='7443', source='1514', initialValue=0.00012423903590508137),
 Row(target='4195', source='1514', initialValue=0.00012423903590508137),
 Row(target='8015', source='1514', initialValue=0.00012423903590508137)]

In [51]:
#mapping page rank weight for each edge
def mapper(df_edge):
    df_edge_source=df_edge.select("source").distinct().rdd.flatMap(lambda x: x).collect()
    df_edge=df_edge.withColumn("pageRank",lit(0))
    for source in df_edge_source:
        #count number of edges under same source node
        count=df_edge.filter((df_edge.source==source)).count()
        #calculate page rank weight under the same source node
        df_edge = df_edge.withColumn("pageRank", when((df_edge.source==source),initial_value/count).otherwise(df_edge.pageRank))
    return df_edge

In [52]:
df_mapper=mapper(df_edge)

In [56]:
df_mapper.filter((df_mapper.source=='401')).show()

+------+------+--------------------+--------------------+
|target|source|        initialValue|            pageRank|
+------+------+--------------------+--------------------+
|  3102|   401|1.242390359050813...|3.300718275905456...|
|  7244|   401|1.242390359050813...|3.300718275905456...|
|  3676|   401|1.242390359050813...|3.300718275905456...|
|  6501|   401|1.242390359050813...|3.300718275905456...|
|    37|   401|1.242390359050813...|3.300718275905456...|
|  3574|   401|1.242390359050813...|3.300718275905456...|
|   492|   401|1.242390359050813...|3.300718275905456...|
|  6978|   401|1.242390359050813...|3.300718275905456...|
|  5956|   401|1.242390359050813...|3.300718275905456...|
|  2168|   401|1.242390359050813...|3.300718275905456...|
|   477|   401|1.242390359050813...|3.300718275905456...|
|    64|   401|1.242390359050813...|3.300718275905456...|
|  4934|   401|1.242390359050813...|3.300718275905456...|
|  4415|   401|1.242390359050813...|3.300718275905456...|
|  6517|   401

In [142]:
def reducer(df):
    df=df.sort(col('target'))
    total = df.groupBy("target").agg(F.sum("pageRank").alias("pageRankTotal"))
    df_as1 = total.alias("df_as1")
    df_as2 = df_node.alias("df_as2")
    df_join = df_as1.join(df_as2, col("df_as1.target") == col("df_as2.label"), 'inner')
    df_join=df_join.select("df_as1.target", "df_as1.pageRankTotal", "df_as2.urls").distinct().sort(col('pageRankTotal').desc())
    return df_join

In [143]:
df_pagerank=reducer(df_mapper)

In [148]:
df_pagerank.show(50,truncate = False)

+------+---------------------+------------------------------+
|target|pageRankTotal        |urls                          |
+------+---------------------+------------------------------+
|6780  |1.6566803333868953E-4|www.xnd.552psb.com            |
|7512  |1.6566803333868953E-4|WWW.CABET710.COM              |
|4595  |1.6566803333868953E-4|WWW.2998.CC                   |
|7566  |6.213550341778839E-5 |0-euro-handys.de              |
|5298  |6.213550341778839E-5 |WWW.SSS0053.COM               |
|7747  |6.213550341778839E-5 |www.24mobile.de               |
|1635  |6.213550341778839E-5 |WWW.BM2109.COM                |
|6766  |3.1075744441518036E-5|WWW.MAIYUNZU.COM              |
|30    |3.105975897627034E-5 |WWW.TY015.COM                 |
|4945  |3.105975897627034E-5 |www.hxn.psb711.com            |
|1112  |3.105975897627034E-5 |www.111922.org                |
|5898  |1.2439889055755833E-5|vpc9y.ssbet241.cn             |
|7564  |1.2439889055755833E-5|so.qhdsfy.com                 |
|4772  |

In [145]:
df_pagerank.count()

8039

In [146]:
df_pagerank.distinct().count()

8039

In [149]:
df_pagerank.write.parquet("s3://aws-logs-780494741284-us-east-1/dataset/df_pagerank",mode="overwrite")

In [150]:
spark.stop()